# Change Detection in Remote Sensing Images using Deep Learning

In [1]:
import os
import numpy as np
from tensorflow import keras
from pyrsgis import raster
from pyrsgis.convert import changeDimension
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score

Warning! matplotlib_scalebar library not found.


In [2]:
!pip install pyrsgis

In [3]:
import os
os.chdir("/content/drive/My Drive/changedetection/Landsat-Classification-Using-Neural-Network-master")

In [4]:
mxHyderabad = "l5_Hyderabad2011_raw.tif"

In [5]:
mxBangalore = 'l5_Bangalore2011_raw.tif'
builtupBangalore = 'l5_Bangalore2011_builtup.tif'

In [6]:
ds1, featuresBangalore = raster.read(mxBangalore, bands='all')
ds2, labelBangalore = raster.read(builtupBangalore, bands=1)

In [7]:
ds3, featuresHyderabad = raster.read(mxHyderabad, bands='all')

In [ ]:
/content/drive/My Drive/changedetection/Landsat-Classification-Using-Neural-Network-master/l5_Hyderabad2011_raw.tif

# Preprocessing

In [8]:
print("Bangalore Multispectral image shape: ", featuresBangalore.shape)
print("Bangalore Binary built-up image shape: ", labelBangalore.shape)
print("Hyderabad Multispectral image shape: ", featuresHyderabad.shape)

Bangalore Multispectral image shape:  (6, 2054, 2044)
Bangalore Binary built-up image shape:  (2054, 2044)
Hyderabad Multispectral image shape:  (6, 1318, 1056)


In [9]:
# Clean the labelled data to replace NoData values by zero
labelBangalore = (labelBangalore == 1).astype(int)

In [10]:
labelBangalore

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [11]:
# Reshape the array to single dimensional array
featuresBangalore = changeDimension(featuresBangalore)
labelBangalore = changeDimension (labelBangalore)
featuresHyderabad = changeDimension(featuresHyderabad)
nBands = featuresBangalore.shape[1]

In [12]:
print("Bangalore Multispectral image shape: ", featuresBangalore.shape)
print("Bangalore Binary built-up image shape: ", labelBangalore.shape)
print("Hyderabad Multispectral image shape: ", featuresHyderabad.shape)

Bangalore Multispectral image shape:  (4198376, 6)
Bangalore Binary built-up image shape:  (4198376,)
Hyderabad Multispectral image shape:  (1391808, 6)


# Training and Testing

In [13]:
# Split testing and training datasets
xTrain, xTest, yTrain, yTest = train_test_split(featuresBangalore, labelBangalore, test_size=0.4, random_state=42)

print(xTrain.shape)
print(yTrain.shape)

print(xTest.shape)
print(yTest.shape)

(2519025, 6)
(2519025,)
(1679351, 6)
(1679351,)


In [14]:
# Normalise the data
xTrain = xTrain / 255.0
xTest = xTest / 255.0
featuresHyderabad = featuresHyderabad / 255.0

# Reshape the data
xTrain = xTrain.reshape((xTrain.shape[0], 1, xTrain.shape[1]))
xTest = xTest.reshape((xTest.shape[0], 1, xTest.shape[1]))
featuresHyderabad = featuresHyderabad.reshape((featuresHyderabad.shape[0], 1, featuresHyderabad.shape[1]))

# Print the shape of reshaped data
print(xTrain.shape, xTest.shape, featuresHyderabad.shape)


(2519025, 1, 6) (1679351, 1, 6) (1391808, 1, 6)


# Building and Running the Model

In [15]:
# Define the parameters of the model
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(1, nBands)),
    keras.layers.Dense(14, activation='relu'),
    keras.layers.Dense(2, activation='softmax')])

In [16]:
model

In [17]:
# Define the accuracy metrics and parameters
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [18]:
# Run the model
model.fit(xTrain, yTrain, epochs=2)

Epoch 1/2
78720/78720 [==============================] - 79s 1ms/step - loss: 0.1233 - accuracy: 0.9537
Epoch 2/2
78720/78720 [==============================] - 79s 1ms/step - loss: 0.1090 - accuracy: 0.9590


# Prediction

In [19]:
# Predict for test data 
yTestPredicted = model.predict(xTest)
yTestPredicted = yTestPredicted[:,1]

# Performance Metrics

In [20]:
# Calculate and display the error metrics
yTestPredicted = (yTestPredicted>0.5).astype(int)
cMatrix = confusion_matrix(yTest, yTestPredicted)
pScore = precision_score(yTest, yTestPredicted)
rScore = recall_score(yTest, yTestPredicted)

In [22]:
print(pScore)
print(rScore)

0.8503221222934462
0.7960146114120167


Confusion Matrix

In [21]:
print("Confusion matrix: for 14 nodes\n", cMatrix)
print("\nP-Score: %.3f, R-Score: %.3f" % (pScore, rScore))

Confusion matrix: for 14 nodes
 [[1453066   27810]
 [  40486  157989]]

P-Score: 0.850, R-Score: 0.796


In [23]:
predicted = model.predict(featuresHyderabad)
predicted = predicted[:,1]

In [24]:
predicted

array([0.06025407, 0.05612851, 0.03471671, ..., 0.00195381, 0.05368536,
       0.9012414 ], dtype=float32)

In [25]:
# Predict new data and export the probability raster
prediction = np.reshape(predicted, (ds3.RasterYSize, ds3.RasterXSize))
outFile = 'Hyderabad_2011_BuiltupNN_predicted.tif'
raster.export(prediction, ds3, filename=outFile, dtype='float')